In [1]:
"""
Relevance Detection
"""

'\nRelevance Detection\n'

In [2]:
import pandas as pd
import nltk
import numpy as np
import preprocessing
import importlib

In [3]:
importlib.reload(preprocessing)

<module 'preprocessing' from '/Users/dannyyang/Documents/GitHub/Insights-FakeNews/preprocessing.py'>

In [4]:
train_stances = pd.read_csv("fn_data/train_stances.csv")
train_stances["Stance"] = train_stances["Stance"].apply(lambda x: "related" if x != "unrelated" else x)
print(train_stances.shape)
train_stances.head()

(49972, 3)


,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,related
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,related


In [5]:
train_bodies = pd.read_csv("fn_data/train_bodies.csv")
print(train_bodies.shape)
train_bodies.head()

(1683, 2)


,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [6]:
preprocessing.get_body(6, train_bodies)

'Posting photos of a gun-toting child online, ISIS supporters announced that the group’s youngest soldier has died in combat.\n\nTwitter accounts linked to the Islamic State of Iraq and Al-Sham claimed that the child soldier “got martyred” with his father while fighting for the terrorist group in Syria.\n\nPhotos posted on Twitter showed the smiling boy in military fatigues holding weapons that, at times, are almost as large as his body. British media reported that the child was roughly 10 years old.\n\nThe photos of the boy first emerged in June, said Charlie Cooper, a researcher who monitors ISIS social media for the London-based Quilliam counter-extremism think tank.\nIn the past week, Mr. Cooper has noticed the hashtag “shibal_alBaghdadi” — which translates as “the cub of Baghdadi” — on Twitter accounts linked to ISIS.\n\nWhile ISIS fighters commonly refer to themselves as lions of the Islamic State, Mr. Cooper said, they refer to child soldiers as cubs of Abu Bakr Al-Baghdadi, ISI